In [1]:
'''Function (Python3): Process Digital Elevation Models (DEMs).

Author: He Zhang @ University of Exeter
Date: Mar 16th 2019 (Update: April 17th 2019)
Contact: hz298@exeter.ac.uk zhangheupc@126.com

DEM Data:
    EUDEMv11  -  EU-DEMv1.1
    Download Link: https://land.copernicus.eu/imagery-in-situ/eu-dem/eu-dem-v1.1

Terms and Abbreviations:
    EPSG  -  European Petroleum Survey Group
    GCS   -  Geographic Coordinate System (Identified by an unique EPSG code)
    PCS   -  Projected Coordinate System (Identified by an unique EPSG code)
    WGS-84 [EPSG 4326]   -  1984 World Geodetic System [GCS]
    Merc [EPSG 3857]     -  Mercator -> Web Mercator -> Pseudo Mercator [PCS of WGS-84]
    OSGB-36 [EPSG 4277]  -  1936 Ordnance Survey Great Britain [GCS]
    BNG [EPSG 27700]     -  British National Grid [PCS of OSGB-36]
    ETRS-89 [EPSG 4258]  -  1989 European Terrestrial Reference System [GCS]
    LAEA [EPSG 3035]     -  Lambert Azimuthal Equal-Area [PCS of ETRS-89]
    lat  -  Latitude
    lng  -  Longitude
    Transform  -  GCS to GCS
    Project    -  GCS to PCS, PCS to PCS
    Convert    -  PCS to PCS

Functions:
    Convert DEM from LAEA PCS to ETRS-89 GCS.

    Transform DEM from ETRS-89 GCS to WGS-84 GCS.
    Transform DEM from ETRS-89 GCS to OSGB-36 GCS.

    Get the elevation from DEM in WGS-84 GCS.
    Get the elevation from DEM in OSGB-36 GCS.
    Get the elevation from DEM in ETRS-89 GCS.

******************** Important Information of Code Usage ********************
- Use 'GDAL.GetProjection()' to check the GCS/PCS information of DEM (in TIF format).
- Use 'GDAL.GetGeoTransform()' to check the resolution of DEMs.
- Use 'GDAL.GetGeoTransform()' to check (lat, lng) of top-left corner of DEM (in GCS).
- The (lat, lng) of other locations in DEM can therefore be calculated.
- Each GCS has one related PCS (GCS/PCS is identified by an unique EPSG code).
- You can transform DEM between different GCSs (e.g., WGS-84 <-> OSGB-36 <-> ETRS-89).
- You can project DEM in GCS to the related PCS and then display its 2D image.
- You can not display DEM in GCS as 2D image (e.g., WGS-84 -> 2D image is wrong).
- You can not project DEM in GCS to the unrelated PCS (e.g., WGS-84 -> BNG is wrong).
- You can not project DEM between different PCSs (e.g., Pseudo Mercator <-> BNG is wrong).
* Use 'gdalwarp' command to transform/project DEM to different GCSs/PCSs might be correct.
'''

# import os
# import re
# import shutil
# import subprocess

# import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
from pandas import read_csv

from pyDEM_function import get_dem_info
from pyDEM_function import get_elevation
# from pyDEM_function import get_file_names
# from pyDEM_function import show_2d_dem
from pyDEM_function import transprojcnvt_dem
# from pyDEM_function import write_dem



In [2]:
# Specify user settings.

# Set the format of DEMs.
DEM_FORMAT = '.tif'

# Set the path of DEMs.
PATH_EUDEM = "DATA/DATA_EUDEMv11/"
# PATH_EUDEM_SOURCE = "DATA/DATA_EUDEMv11/EPSG3035_s/"  # The folder of source DEMs must exist.

# Set the name of DEMs in GCSs.
EUDEM_GCS_WD = "EUDEMv11_EPSG4326.tif"
EUDEM_GCS_UK = "EUDEMv11_EPSG4277.tif"
EUDEM_GCS_EU = "EUDEMv11_EPSG4258.tif"

# Set the name of DEMs in PCSs.
EUDEM_PCS_WD = "EUDEMv11_EPSG3857.tif"
EUDEM_PCS_UK = "EUDEMv11_EPSG27700.tif"
EUDEM_PCS_EU = "EUDEMv11_EPSG3035.tif"

# Set the path of location data file.
PATH_LD_STATION_DATA = "DATA/DATA_LD_AirQuality/London_AirQuality_Stations.csv"


In [3]:
# <EUDEMv11> Convert DEM from LAEA PCS to ETRS-89 GCS.

print('\n>>> <EUDEMv11> Convert DEM from LAEA PCS to ETRS-89 GCS.')

path = PATH_EUDEM
dem_in = path + EUDEM_PCS_EU
dem_out = path + EUDEM_GCS_EU

epsg_in = 3035
epsg_out = 4258

transprojcnvt_dem(dem_in, epsg_in, dem_out, epsg_out)

data = gdal.Open(dem_in)
get_dem_info(data, 1)
data = gdal.Open(dem_out)
get_dem_info(data, 1)

print('\n>>> Complete!\n')



>>> <EUDEMv11> Convert DEM from LAEA PCS to ETRS-89 GCS.

The information of DEM:
The number of row (height) is: 40000
The number of column (width) is: 40000
The number of band is: 1
The 6 GeoTransform parameters are:
 (3000000.0, 25.0, 0.0, 4000000.0, 0.0, -25.0)
The GCS/PCS information is:
 PROJCS["ETRS89_ETRS_LAEA",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.2572221010002,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4258"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",52],PARAMETER["longitude_of_center",10],PARAMETER["false_easting",4321000],PARAMETER["false_northing",3210000],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]

The information of DEM:
The number of row (height) is: 30505
The number of column (width) is: 52428
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266271034206424, 0.0003392098021871587

In [4]:
# <EUDEMv11> Transform DEM from ETRS-89 GCS to WGS-84 GCS.

print('\n>>> <EUDEMv11> Transform DEM from ETRS-89 GCS to WGS-84 GCS.')

path = PATH_EUDEM
dem_in = path + EUDEM_GCS_EU
dem_out = path + EUDEM_GCS_WD

epsg_in = 4258
epsg_out = 4326

transprojcnvt_dem(dem_in, epsg_in, dem_out, epsg_out)

data = gdal.Open(dem_in)
get_dem_info(data, 1)
data = gdal.Open(dem_out)
get_dem_info(data, 1)

print('\n>>> Complete!\n')



>>> <EUDEMv11> Transform DEM from ETRS-89 GCS to WGS-84 GCS.

The information of DEM:
The number of row (height) is: 30505
The number of column (width) is: 52428
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266271034206424, 0.00033920980218715875, 0.0, 58.98761768429964, 0.0, -0.00033920980218715875)
The GCS/PCS information is:
 GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.2572221010002,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4258"]]

The information of DEM:
The number of row (height) is: 30505
The number of column (width) is: 52428
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266271034206424, 0.00033920980218715875, 0.0, 58.98761768429964, 0.0, -0.00033920980218715875)
The GCS/PCS information is:
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EP

In [5]:
# <EUDEMv11> Transform DEM from ETRS-89 GCS to OSGB-36 GCS.

print('\n>>> <EUDEMv11> Transform DEM from ETRS-89 GCS to OSGB-36 GCS.')

path = PATH_EUDEM
dem_in = path + EUDEM_GCS_EU
dem_out = path + EUDEM_GCS_UK

epsg_in = 4258
epsg_out = 4277

transprojcnvt_dem(dem_in, epsg_in, dem_out, epsg_out)

data = gdal.Open(dem_in)
get_dem_info(data, 1)
data = gdal.Open(dem_out)
get_dem_info(data, 1)

print('\n>>> Complete!\n')



>>> <EUDEMv11> Transform DEM from ETRS-89 GCS to OSGB-36 GCS.

The information of DEM:
The number of row (height) is: 30505
The number of column (width) is: 52428
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266271034206424, 0.00033920980218715875, 0.0, 58.98761768429964, 0.0, -0.00033920980218715875)
The GCS/PCS information is:
 GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.2572221010002,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4258"]]

The information of DEM:
The number of row (height) is: 30506
The number of column (width) is: 52430
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266102928602475, 0.0003392475335051166, 0.0, 58.98812292836723, 0.0, -0.0003392475335051166)
The GCS/PCS information is:
 GEOGCS["OSGB 1936",DATUM["OSGB_1936",SPHEROID["Airy 1830",6377563.396,299.3249646000044

In [6]:
# Read London air quality monitoring station data file.

print('\n>>> Read London air quality monitoring station data file.')

site_data = read_csv(PATH_LD_STATION_DATA)
print(site_data.head(3))

site_num = site_data['SiteName'].count()
print('\n*==> The number of stations is: %d' % site_num)

# Get the latitude and longitude of stations.
site_latlng = np.zeros((site_num, 2))
site_latlng[:, 0] = site_data['Latitude']  # 0-th column - Latitude.
site_latlng[:, 1] = site_data['Longitude']  # 1-th column - Longitude.

np.set_printoptions(suppress=True)  # Print numbers without scientific notation.
print('\n*==> The location (lat, lng) of stations are:\n', site_latlng)

print('\n>>> Complete!\n')



>>> Read London air quality monitoring station data file.
  Unnamed: 0 api_data need_prediction  historical_data   Latitude  Longitude  \
0        BX9     True             NaN             True  51.465983   0.184877   
1        BX1     True             NaN             True  51.465983   0.184877   
2        BL0     True            True             True  51.522287  -0.125848   

           SiteType                   SiteName  
0          Suburban  Bexley - Slade Green FDMS  
1          Suburban       Bexley - Slade Green  
2  Urban Background        Camden - Bloomsbury  

*==> The number of stations is: 24

*==> The location (lat, lng) of stations are:
 [[51.46598327  0.18487713]
 [51.46598327  0.18487713]
 [51.522287   -0.125848  ]
 [51.52770662 -0.12905321]
 [51.544219   -0.175284  ]
 [51.51452534 -0.10451563]
 [51.51384718 -0.07776568]
 [51.410039   -0.127523  ]
 [51.490532    0.074003  ]
 [51.45258     0.070766  ]
 [51.486957    0.095111  ]
 [51.456357    0.040725  ]
 [51.4563      0

In [7]:
# <EUDEMv11> Get the elevation from DEM in WGS-84 GCS.

print('\n>>> <EUDEMv11> Get the elevation from DEM in WGS-84 GCS.')

dem_gcs = gdal.Open(PATH_EUDEM + EUDEM_GCS_WD)
get_dem_info(dem_gcs, 1)

site_ele_eudem_wgs = get_elevation(dem_gcs, site_latlng)
np.set_printoptions(suppress=True)

print('\n*==> The elevation information of stations is:\n', site_ele_eudem_wgs)
print('\n*==> The elevation value of stations is:\n', site_ele_eudem_wgs[:, 5].astype(int))

print('\n>>> Complete!\n')



>>> <EUDEMv11> Get the elevation from DEM in WGS-84 GCS.

The information of DEM:
The number of row (height) is: 30505
The number of column (width) is: 52428
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266271034206424, 0.00033920980218715875, 0.0, 58.98761768429964, 0.0, -0.00033920980218715875)
The GCS/PCS information is:
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]

The 6 GeoTransform parameters of DEM are:
 (-12.266271034206424, 0.00033920980218715875, 0.0, 58.98761768429964, 0.0, -0.00033920980218715875)

*==> The elevation information of stations is:
 [[    0.            51.46598327     0.18487713 22174.
  36706.            12.63551998]
 [    1.            51.46598327     0.18487713 22174.
  36706.            12.63551998]
 [    2.            51.522287      -0.125848   22008.
  35790.            37.

In [8]:
# <EUDEMv11> Get the elevation from DEM in OSGB-36 GCS.

print('\n>>> <EUDEMv11> Get the elevation from DEM in OSGB-36 GCS.')

dem_gcs = gdal.Open(PATH_EUDEM + EUDEM_GCS_UK)
get_dem_info(dem_gcs, 1)

site_ele_eudem_osgb = get_elevation(dem_gcs, site_latlng)
np.set_printoptions(suppress=True)

print('\n*==> The elevation information of stations is:\n', site_ele_eudem_osgb)
print('\n*==> The elevation value of stations is:\n', site_ele_eudem_osgb[:, 5].astype(int))

print('\n>>> Complete!\n')



>>> <EUDEMv11> Get the elevation from DEM in OSGB-36 GCS.

The information of DEM:
The number of row (height) is: 30506
The number of column (width) is: 52430
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266102928602475, 0.0003392475335051166, 0.0, 58.98812292836723, 0.0, -0.0003392475335051166)
The GCS/PCS information is:
 GEOGCS["OSGB 1936",DATUM["OSGB_1936",SPHEROID["Airy 1830",6377563.396,299.3249646000044,AUTHORITY["EPSG","7001"]],TOWGS84[446.448,-125.157,542.06,0.15,0.247,0.842,-20.489],AUTHORITY["EPSG","6277"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4277"]]

The 6 GeoTransform parameters of DEM are:
 (-12.266102928602475, 0.0003392475335051166, 0.0, 58.98812292836723, 0.0, -0.0003392475335051166)

*==> The elevation information of stations is:
 [[    0.            51.46598327     0.18487713 22173.
  36702.            12.47727203]
 [    1.            51.46598327     0.18487713 22173.
  36702.            12.47727203]
 [    2

In [9]:
# <EUDEMv11> Get the elevation from DEM in ETRS-89 GCS.

print('\n>>> <EUDEMv11> Get the elevation from DEM in ETRS-89 GCS.')

dem_gcs = gdal.Open(PATH_EUDEM + EUDEM_GCS_EU)
get_dem_info(dem_gcs, 1)

site_ele_eudem_etrs = get_elevation(dem_gcs, site_latlng)
np.set_printoptions(suppress=True)

print('\n*==> The elevation information of stations is:\n', site_ele_eudem_etrs)
print('\n*==> The elevation value of stations is:\n', site_ele_eudem_etrs[:, 5].astype(int))

print('\n>>> Complete!\n')



>>> <EUDEMv11> Get the elevation from DEM in ETRS-89 GCS.

The information of DEM:
The number of row (height) is: 30505
The number of column (width) is: 52428
The number of band is: 1
The 6 GeoTransform parameters are:
 (-12.266271034206424, 0.00033920980218715875, 0.0, 58.98761768429964, 0.0, -0.00033920980218715875)
The GCS/PCS information is:
 GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.2572221010002,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4258"]]

The 6 GeoTransform parameters of DEM are:
 (-12.266271034206424, 0.00033920980218715875, 0.0, 58.98761768429964, 0.0, -0.00033920980218715875)

*==> The elevation information of stations is:
 [[    0.            51.46598327     0.18487713 22174.
  36706.            12.63551998]
 [    1.            51.46598327     0.18487713 22174.
  36706.            12.63551998]
 [    2.     

In [10]:
# Compare the elevation obtained from different DEMs.

print('\n>>> Compare the elevation obtained from different DEMs.')

print('\nEUDEMv11_WD', site_ele_eudem_wgs[:, 5].astype(int))
print('\nEUDEMv11_UK', site_ele_eudem_osgb[:, 5].astype(int))
print('\nEUDEMv11_EU', site_ele_eudem_etrs[:, 5].astype(int))

print('\n>>> Complete!\n')



>>> Compare the elevation obtained from different DEMs.

EUDEMv11_WD [12 12 37 30 61 25 29 36 11 66 11 31 64 79  7 27 25 25 14 15 13 32  5 36]

EUDEMv11_UK [12 12 37 26 58 30 32 35 11 71  8 30 66 78  8 26 24 24  8 16 13 32  9 35]

EUDEMv11_EU [12 12 37 30 61 25 29 36 11 66 11 31 64 79  7 27 25 25 14 15 13 32  5 36]

>>> Complete!

